Initialization
----------------
First, we load alto and import all relevant packages for convenience.  Replace the version of the jar below with the current version.

In [1]:
interp.load.cp(os.Path(System.getProperty("user.dir")+"/build/libs/alto-2.3.5-SNAPSHOT-all.jar"))

In [2]:
import de.up.ling.irtg._
import de.up.ling.irtg.algebra._
import de.up.ling.irtg.automata._
import de.up.ling.irtg.hom._
import de.up.ling.irtg.signature._
import de.up.ling.irtg.corpus._
import de.up.ling.irtg.codec._
import de.up.ling.irtg.util.FirstOrderModel
import de.up.ling.tree._
import de.saar.basic._
import de.up.ling.irtg.util.TestingTools._

import java.io._
import collection.JavaConverters._

import scala.language.implicitConversions;

implicit def intmap2integermap(map:scala.collection.immutable.Map[String,Int]) = map.asJava.asInstanceOf[java.util.Map[String,java.lang.Integer]]
implicit def stringmap2java(map:scala.collection.immutable.Map[String,String]) = map.asJava
implicit def string2tree(s:String) = pt(s)


import de.up.ling.irtg._

import de.up.ling.irtg.algebra._

import de.up.ling.irtg.automata._

import de.up.ling.irtg.hom._

import de.up.ling.irtg.signature._

import de.up.ling.irtg.corpus._

import de.up.ling.irtg.codec._

import de.up.ling.irtg.util.FirstOrderModel

import de.up.ling.tree._

import de.saar.basic._

import de.up.ling.irtg.util.TestingTools._


import java.io._

import collection.JavaConverters._


import scala.language.implicitConversions;


defined function intmap2integermap
defined function stringmap2java
defined function string2tree

Interpreted regular tree grammars
--------------------------------------

IRTGs are loaded by an input codec object. A codec reads IRTGs from input streams and generates an InterpretedTreeAutomaton representing the irtg.

In [3]:
val ic = new IrtgInputCodec()
val fis = new FileInputStream("examples/scfg.irtg")
val irtg = ic.read(fis)

ic: IrtgInputCodec = de.up.ling.irtg.codec.IrtgInputCodec@394b7504
fis: FileInputStream = java.io.FileInputStream@677bdb4
irtg: InterpretedTreeAutomaton = interpretation english: de.up.ling.irtg.algebra.StringAlgebra
interpretation german: de.up.ling.irtg.algebra.StringAlgebra

N -> r11 [1.0]
  [english] telescope
  [german] fernrohr

P -> r12 [1.0]
  [english] with
  [german] mit

V -> r8 [1.0]
  [english] watches
  [german] betrachtet

Det -> r9 [1.0]
  [english] the
  [german] die

N -> r10 [1.0]
  [english] woman
  [german] frau

Det -> r9b [1.0]
  [english] the
  [german] dem

NP -> r7 [1.0]
  [english] john
  [german] hans

S! -> r1(NP, VP) [1.0]
  [english] *(?1,?2)
  [german] *(?1,?2)

NP -> r2(Det, N) [1.0]
  [english] *(?1,?2)
  [german] *(?1,?2)

...

An irtg can be used in different ways. parsing results in a chart:

In [4]:
val chart = irtg.parse(Map("german" -> "hans betrachtet die frau mit dem fernrohr"))

chart: TreeAutomaton[?0] = 'N,6-7' -> r11 [1.0]
'P,4-5' -> r12 [1.0]
'V,1-2' -> r8 [1.0]
'Det,2-3' -> r9 [1.0]
'N,3-4' -> r10 [1.0]
'Det,5-6' -> r9b [1.0]
'NP,0-1' -> r7 [1.0]
'VP,1-7' -> r4('V,1-2', 'NP,2-7') [1.0]
'VP,1-4' -> r4('V,1-2', 'NP,2-4') [1.0]
'NP,5-7' -> r2('Det,5-6', 'N,6-7') [1.0]
'N,3-7' -> r3('N,3-4', 'PP,4-7') [1.0]
'VP,1-7' -> r5('VP,1-4', 'PP,4-7') [1.0]
'S,0-7'! -> r1('NP,0-1', 'VP,1-7') [1.0]
'S,0-4' -> r1('NP,0-1', 'VP,1-4') [1.0]
'NP,2-4' -> r2('Det,2-3', 'N,3-4') [1.0]
'NP,2-7' -> r2('Det,2-3', 'N,3-7') [1.0]
'PP,4-7' -> r6('P,4-5', 'NP,5-7') [1.0]


The chart can then be used for viterbi decoding, resulting in a derivation tree:

In [5]:
chart.viterbi

res4: Tree[String] = r1(r7,r5(r4(r8,r2(r9,r10)),r6(r12,r2(r9b,r11))))

Alternatively, the irtg can be used to decode, i.e. to transfer from one interpretation to another.  In this case, the target is the english interpretation and the source is the german interpretation with a specific interpretation given.

In [6]:
irtg.decode("english", Map("german" -> "hans betrachtet die frau mit dem fernrohr"))

res5: java.util.Set[?0] = [[john, watches, the, woman, with, the, telescope]]

Algebras
---------

Algebras define operations generating an interpretation.  Their operations are a homomorphism of the derivation operations.

Given an interpretation, an algebra can generate a chart describing the possible ways this interpretations could hav been constructed:

In [7]:
val alg = new StringAlgebra()
val ta_abc = alg.decompose(alg.parseString("a b c"))

alg: StringAlgebra = de.up.ling.irtg.algebra.StringAlgebra@3c338b7b
ta_abc: TreeAutomaton[State] = '1-2' -> b [1.0]
'0-1' -> a [1.0]
'2-3' -> c [1.0]
'0-2' -> *('0-1', '1-2') [1.0]
'0-3'! -> *('0-1', '1-3') [1.0]
'1-3' -> *('1-2', '2-3') [1.0]
'0-3'! -> *('0-2', '2-3') [1.0]


In [8]:
val ta_xyz = alg.decompose(alg.parseString("x y z"))

ta_xyz: TreeAutomaton[State] = '0-1' -> x [1.0]
'2-3' -> z [1.0]
'1-2' -> y [1.0]
'0-2' -> *('0-1', '1-2') [1.0]
'0-3'! -> *('0-1', '1-3') [1.0]
'1-3' -> *('1-2', '2-3') [1.0]
'0-3'! -> *('0-2', '2-3') [1.0]


In [9]:
ta_abc.intersect(ta_xyz)

res8: TreeAutomaton[Pair[State, State]] = 

In [10]:
val ta_abc2 = alg.decompose(alg.parseString("a b c"))
ta_abc.intersect(ta_abc2)

ta_abc2: TreeAutomaton[State] = '1-2' -> b [1.0]
'0-1' -> a [1.0]
'2-3' -> c [1.0]
'1-3' -> *('1-2', '2-3') [1.0]
'0-2' -> *('0-1', '1-2') [1.0]
'0-3'! -> *('0-1', '1-3') [1.0]
'0-3'! -> *('0-2', '2-3') [1.0]

res9_1: TreeAutomaton[Pair[State, State]] = '1-2,1-2' -> b [1.0]
'0-1,0-1' -> a [1.0]
'2-3,2-3' -> c [1.0]
'0-2,0-2' -> *('0-1,0-1', '1-2,1-2') [1.0]
'0-3,0-3'! -> *('0-1,0-1', '1-3,1-3') [1.0]
'0-3,0-3'! -> *('0-2,0-2', '2-3,2-3') [1.0]
'1-3,1-3' -> *('1-2,1-2', '2-3,2-3') [1.0]


IRTGs
-----
An IRTG consists of a definition of derivation trees, a set of algebras and
a homomorphism from the derivations to each algebra.
The file format first has a definition of the interpretations and
then a set of rules, each of which are followed by a corresponding interpretation for each algebra.

input codecs are responsible for reading irtgs, we will read an example consisting of two interpretations: A string algebra mapping to sentences and a graph algebra mapping to an AMR representation.

In [11]:
def irtgFromFile(fname: String): InterpretedTreeAutomaton = {
  val ic = new IrtgInputCodec()
  ic.read(new FileInputStream(fname))
}

defined function irtgFromFile

In [12]:
val irtgStringGraph = irtgFromFile("examples/altoDemoStringToGraph.irtg")

irtgStringGraph: InterpretedTreeAutomaton = interpretation english: de.up.ling.irtg.algebra.StringAlgebra
interpretation semantic: de.up.ling.irtg.algebra.graph.GraphAlgebra

NP -> r4 [1.0]
  [english] *(the,news)
  [semantic] '(u<obj2> / news)'

NE -> r2 [0.6]
  [english] John
  [semantic] '(u<root> / John)'

NE -> r3 [0.4]
  [english] Mary
  [semantic] '(u<root> / Mary)'

S! -> r1(NE, NE, NP) [1.0]
  [english] *(?1,*(tells,*(?2,?3)))
  [semantic] merge(merge(merge('(u<root> / tell :ARG0 (s<subj>) :ARG1 (o1<obj1>) :ARG2 (o2<obj2>))',?3),r_root_obj1(?2)),r_root_subj(?1))



In [13]:
val ta = irtgStringGraph.parse(Map("english" -> "John tells Mary the news"))

ta: TreeAutomaton[?0] = 'NP,3-5' -> r4 [1.0]
'NE,0-1' -> r2 [0.6]
'NE,2-3' -> r3 [0.4]
'S,0-5'! -> r1('NE,0-1', 'NE,2-3', 'NP,3-5') [1.0]


We can now decode a sentence by supplying an input for one interpretation and generating all interpretations of another algebra consistent with this input.

In [14]:
irtgStringGraph.decode("semantic", Map("english" -> "John tells Mary the news"))
irtgStringGraph.decode("semantic", Map("english" -> "Mary tells John the news"))

res13_0: java.util.Set[?0] = [[u_4<root>/tell -ARG0-> u_2<subj>/John; u_4 -ARG1-> u_1<obj1>/Mary; u_4 -ARG2-> u_3<obj2>/news]]
res13_1: java.util.Set[?0] = [[u_11<root>/tell -ARG0-> u_9<subj>/Mary; u_11 -ARG1-> u_8<obj1>/John; u_11 -ARG2-> u_10<obj2>/news]]

Weighted tree grammars
-------------------------
Derivation in tree grammars may be weighted.  These allow the derivation process to not just pick any derivation, but the best one according to the weights.

With these, PCFGs can be implemented among others.  We will now load a toy grammar, parse a sentence into a chart and use that chart to obtain the most probable structure.

In [15]:
val elephant = irtgFromFile("examples/elephant-weighted.irtg")

elephant: InterpretedTreeAutomaton = interpretation i: de.up.ling.irtg.algebra.StringAlgebra

NP -> r2 [0.3]
  [i] john

NP -> r4 [0.2]
  [i] *(his,pyjamas)

V -> r8 [1.0]
  [i] shot

P -> r10 [1.0]
  [i] in

NP -> r3 [0.2]
  [i] *(an,elephant)

NP -> r5(NP, PP) [0.3]
  [i] *(?1,?2)

S! -> r1(NP, VP) [1.0]
  [i] *(?1,?2)

PP -> r9(P, NP) [1.0]
  [i] *(?1,?2)

VP -> r7(VP, PP) [0.5]
  [i] *(?1,?2)

VP -> r6(V, NP) [0.5]
  [i] *(?1,?2)



In [16]:
val chart = elephant.parse(Map("i"->"john shot an elephant in his pyjamas"))

chart: TreeAutomaton[?0] = 'NP,0-1' -> r2 [0.3]
'NP,5-7' -> r4 [0.2]
'V,1-2' -> r8 [1.0]
'P,4-5' -> r10 [1.0]
'NP,2-4' -> r3 [0.2]
'VP,1-4' -> r6('V,1-2', 'NP,2-4') [0.5]
'VP,1-7' -> r6('V,1-2', 'NP,2-7') [0.5]
'NP,2-7' -> r5('NP,2-4', 'PP,4-7') [0.3]
'VP,1-7' -> r7('VP,1-4', 'PP,4-7') [0.5]
'S,0-4' -> r1('NP,0-1', 'VP,1-4') [1.0]
'S,0-7'! -> r1('NP,0-1', 'VP,1-7') [1.0]
'PP,4-7' -> r9('P,4-5', 'NP,5-7') [1.0]


We now have a chart with weights.  This chart is stored in a TreeAutomaton object, which allows to perform viterbi decoding.  viterbiRaw gives us a WeightedTree -- a tree consisting of the internal ids of the rules and a weight for this tree.  But let us have a look at the language this chart encode first.  The language consists of two trees: 

In [17]:
chart.language()

res16: java.util.Set[Tree[String]] = [r1(r2,r6(r8,r5(r3,r9(r10,r4)))), r1(r2,r7(r6(r8,r3),r9(r10,r4)))]

In [18]:
val tree = chart.viterbiRaw()

tree: WeightedTree = '1'('2','7'('6'('8','3'),'9'('10','4'))):0.0030000000000000005

Alternatively, we can call viterbi() and obtain a tree consisting of the rule names as strings:

In [19]:
val stree = chart.viterbi()

stree: Tree[String] = r1(r2,r7(r6(r8,r3),r9(r10,r4)))

Querying for an interpretations of this tree gives us the sentence we originally used to construct the chart, no surprise there:

In [20]:
elephant.interpret(stree, "i")

res19: Object = [john, shot, an, elephant, in, his, pyjamas]

IRTGs with multiple interpretations
===================
We now use the example grammar from Koller (2015), Semantic construction with graph grammars.  Instead of one interpretation (a string interpretation representing the sentence) we now have two interpretations:

 - a string interpretation representing the sentence and
 - a graph interpretation representing the semantics as an s-graph.

Let's load the grammar and see what we can do.

In [21]:
val iwcs = irtgFromFile("examples/hrg-iwcs15.irtg")
val boyChart = iwcs.parse(Map("string" -> "the boy wants to sleep"))

iwcs: InterpretedTreeAutomaton = interpretation string: de.up.ling.irtg.algebra.StringAlgebra
interpretation graph: de.up.ling.irtg.algebra.graph.GraphAlgebra

VP -> sleep [1.0]
  [string] sleep
  [graph] '(g<root> / sleep  :ARG0 (s<subj>))'

NP -> boy [1.0]
  [string] *(the,boy)
  [graph] '(x<root> / boy)'

VP -> go [1.0]
  [string] go
  [graph] '(g<root> / go  :ARG0 (s<subj>))'

VP -> snore [1.0]
  [string] snores
  [graph] '(u<root> / snore :ARG0 (v<subj>))'

RP -> who [1.0]
  [string] who
  [graph] '(u<root>)'

VP -> sleeps [1.0]
  [string] sleeps
  [graph] '(g<root> / sleep  :ARG0 (s<subj>))'

NP -> girl [1.0]
  [string] *(the,girl)
  [graph] '(x<root> / girl)'

VP -> goes [1.0]
  [string] goes
  [graph] '(g<root> / go  :ARG0 (s<subj>))'

NP -> mod_rc(NP, RC) [1.0]
  [string] *(?1,?2)
  [graph] merge(?1,?2)

...
boyChart: TreeAutomaton[?0] = 'VP,4-5' -> sleep [1.0]
'NP,0-2' -> boy [1.0]
'VP,2-5' -> want1('VP,4-5') [1.0]
'S,0-5'! -> comb_subj('NP,0-2', 'VP,2-5') [1.0]


In [22]:
val boyTree = boyChart.viterbi()

boyTree: Tree[String] = comb_subj(boy,want1(sleep))

In [23]:
iwcs.interpret(boyTree)

res22: java.util.Map[String, Object] = {string=[the, boy, wants, to, sleep], graph=[u_16<root>/want -ARG0-> u_15/boy; u_16 -ARG1-> u_17/sleep; u_17 -ARG0-> u_15]}

Nothing really new sofar, except that the interpretation of a derivation tree now gives us both a string (the sentence) and a graph (the semantics).

The whole IRTG approach is different from other formalism because the sentence representation is not special but just another interpretation of the derivation.  We could just as well have started from the semantics interpretation and then go to the string (i.e. sentence) interpretation from there:

In [24]:
val ta = iwcs.parse(Map("graph" -> "(u_35 / sleep  :ARG0 (u_36 / girl)  :ARG1-of (u_34 / want  :ARG0 (u_33 / boy)))"))

ta: TreeAutomaton[?0] = 'VP,[u_35<root> {u_35_u_36, u_35_u_35}, u_36<subj> {u_35_u_36}]' -> sleep [1.0]
'NP,[u_33<root> {u_33_u_33}]' -> boy [1.0]
'RP,[u_34<root> {}]' -> who [1.0]
'RP,[u_35<root> {}]' -> who [1.0]
'RP,[u_36<root> {}]' -> who [1.0]
'RP,[u_33<root> {}]' -> who [1.0]
'VP,[u_35<root> {u_35_u_36, u_35_u_35}, u_36<subj> {u_35_u_36}]' -> sleeps [1.0]
'NP,[u_36<root> {u_36_u_36}]' -> girl [1.0]
'S,[u_34<root> {u_34_u_35, u_34_u_34, u_34_u_33}]'! -> comb_subj('NP,[u_33<root> {u_33_u_33}]', 'VP,[u_33<subj> {u_34_u_33}, u_34<root> {u_34_u_35, u_34_u_34, u_34_u_33}]') [1.0]
'NP,[u_33<root> {u_34_u_33, u_33_u_33}]' -> mod_rc('NP,[u_33<root> {u_33_u_33}]', 'RC,[u_33<root> {u_34_u_33}]') [1.0]
'VP,[u_33<subj> {u_34_u_33}, u_34<root> {u_34_u_35, u_34_u_34, u_34_u_33}]' -> want2('NP,[u_36<root> {u_36_u_36}]', 'VP,[u_35<root> {u_35_u_36, u_35_u_35}, u_36<subj> {u_35_u_36}]') [1.0]
'S,[u_35<root> {u_35_u_36, u_35_u_35}]' -> comb_subj('NP,[u_36<root> {u_36_u_36}]', 'VP,[u_35<root> {u_35_

In [25]:
val lang = ta.language()

lang: java.util.Set[Tree[String]] = [comb_subj(boy,want2(girl,sleeps)), comb_subj(boy,want2(girl,sleep))]

In [26]:
lang.asScala.map(iwcs.interpret(_, "string"))

res25: collection.mutable.Set[Object] = Set(
  [the, boy, wants, the, girl, to, sleeps],
  [the, boy, wants, the, girl, to, sleep]
)

So these are the two sentences that could have generated the semantic used as input.  This IRTG can now both be used for going from sentence to semantics (i.e. infer meaning) and for going from semantics to sentence (i.e. to perform natural language generation).

Template IRTGs and set algebra
================
template IRTGs contain derivations which are parametrized over a set of objects and relations, e.g. if you want to talk about buttons, have several of them but don't want to write the essentially the same rule over and over again.  Instantiating a TIRTG results in a normal IRTG.

In [27]:
def tirtgFromFile(fname: String): TemplateInterpretedTreeAutomaton = {
  val ic = new TemplateIrtgInputCodec()
  ic.read(new FileInputStream(fname))
}

defined function tirtgFromFile

A model for a template is encoded as a json string.  This model defines three buttons and several predicates (button, round, square, left-of and right-of).  The relations are used to determine which template expansions take place.  A template rule looks like this:

    foreach {a | square(a)}:
    N_$a -> square_$a(N_$a)  
    [sem] intersect_1(square, ?1)
    [string] *(square, ?1)


The foreach part determines that only entities for which the predicate square is in the model will be used for expansion.  For each of these entities, a rule is created by replacing `$a` with the name of that entity.

In [28]:
val irtgModel = """{ "button" : [["b1"], ["b2"], ["b3"]],
"round" : [["b1"], ["b3"]],
"square": [["b2"]],
"left-of": [["b1","b2"], ["b2","b3"]],
"right-of": [["b3","b2"], ["b2", "b1"]]}
"""

irtgModel: String = """{ "button" : [["b1"], ["b2"], ["b3"]],
"round" : [["b1"], ["b3"]],
"square": [["b2"]],
"left-of": [["b1","b2"], ["b2","b3"]],
"right-of": [["b3","b2"], ["b2", "b1"]]}
"""

In [29]:
val sigT = tirtgFromFile("examples/sig.tirtg")

sigT: TemplateInterpretedTreeAutomaton = interpretation string: de.up.ling.irtg.algebra.StringAlgebra
interpretation sem: de.up.ling.irtg.algebra.SetAlgebra
[a] button[a]
N_$a -> button_$a[]

[a] blue[a]
N_$a -> blue_$a[N_$a]

[a] square[a]
N_$a -> square_$a[N_$a]

[a] green[a]
N_$a -> green_$a[N_$a]

[a] red[a]
N_$a -> red_$a[N_$a]

[a] yellow[a]
N_$a -> yellow_$a[N_$a]

[a] round[a]
N_$a -> round_$a[N_$a]

[a, b] right-of[a, b]
N_$a -> rightof_$a_$b[N_$a, NP_$b]

[a, b] left-of[a, b]
N_$a -> leftof_$a_$b[N_$a, NP_$b]

[a] T[a]
NP_$a ! -> def_$a[N_$a]



In [30]:
val fom = FirstOrderModel.read(new StringReader(irtgModel))
val sigi = sigT.instantiate(fom)

fom: FirstOrderModel = de.up.ling.irtg.util.FirstOrderModel@5a0f132e
sigi: InterpretedTreeAutomaton = interpretation string: de.up.ling.irtg.algebra.StringAlgebra
interpretation sem: de.up.ling.irtg.algebra.SetAlgebra

N_b3 -> button_b3 [1.0]
  [string] button
  [sem] button

N_b2 -> button_b2 [1.0]
  [string] button
  [sem] button

N_b1 -> button_b1 [1.0]
  [string] button
  [sem] button

NP_b3! -> def_b3(N_b3) [1.0]
  [string] *(the,?1)
  [sem] member_b3(?1)

N_b3 -> round_b3(N_b3) [1.0]
  [string] *(round,?1)
  [sem] intersect_1(round,?1)

N_b3 -> rightof_b3_b2(N_b3, NP_b2) [1.0]
  [string] *(?1,*('right of',?2))
  [sem] project_1(intersect_2(intersect_1(right-of,?1),?2))

N_b2 -> square_b2(N_b2) [1.0]
  [string] *(square,?1)
  [sem] intersect_1(square,?1)

NP_b2! -> def_b2(N_b2) [1.0]
  [string] *(the,?1)
  [sem] member_b2(?1)

N_b2 -> leftof_b2_b3(N_b2, NP_b3) [1.0]
  [string] *(?1,*('left of',?2))
  [sem] project_1(intersect_2(intersect_1(left-of,?1),?2))

...

Now that the grammar is instantiated, we still need to tell the semantic interpretation about the model.  Otherwise it will be unable to create an interpretation.

In [31]:
val alg = sigi.getInterpretation("sem").getAlgebra()
alg.asInstanceOf[SetAlgebra].setModel(fom)

alg: Algebra[?0] = de.up.ling.irtg.algebra.SetAlgebra@21532d32

Everything is set up, the irtg can finally be used!
Let's see what "the square button" and "the button" could refer to:

In [32]:
sigi.decode("sem", Map("string" -> "the square button"))
sigi.decode("sem", Map("string" -> "the button"))

res31_0: java.util.Set[?0] = [[[b2]]]
res31_1: java.util.Set[?0] = [[[b3]], [[b1]], [[b2]]]

Again: No surprise.  There is only one square button in our model but "the button" could refer to any button.
As the instantiated TIRTG is just an IRTG as the others used above, we can again create a chart, get the language of a chart and so on.

In [33]:
val ta = sigi.parse(Map("string" -> "the square button"))

ta: TreeAutomaton[?0] = 'N_b3,2-3' -> button_b3 [1.0]
'N_b2,2-3' -> button_b2 [1.0]
'N_b1,2-3' -> button_b1 [1.0]
'N_b2,1-3' -> square_b2('N_b2,2-3') [1.0]
'NP_b2,0-3'! -> def_b2('N_b2,1-3') [1.0]


In [34]:
ta.language()

res33: java.util.Set[Tree[String]] = [def_b2(square_b2(button_b2))]

In [35]:
val sigia = sigi.getAutomaton()

sigia: TreeAutomaton[String] = N_b3 -> button_b3 [1.0]
N_b2 -> button_b2 [1.0]
N_b1 -> button_b1 [1.0]
NP_b3! -> def_b3(N_b3) [1.0]
N_b3 -> round_b3(N_b3) [1.0]
N_b3 -> rightof_b3_b2(N_b3, NP_b2) [1.0]
N_b2 -> square_b2(N_b2) [1.0]
NP_b2! -> def_b2(N_b2) [1.0]
N_b2 -> leftof_b2_b3(N_b2, NP_b3) [1.0]
N_b2 -> rightof_b2_b1(N_b2, NP_b1) [1.0]
N_b1 -> round_b1(N_b1) [1.0]
NP_b1! -> def_b1(N_b1) [1.0]
N_b1 -> leftof_b1_b2(N_b1, NP_b2) [1.0]


In [36]:
val li = sigia.languageIterator()

li: java.util.Iterator[Tree[String]] = com.google.common.collect.Iterators$6@793fc4b1

Don't do something like this: `sigi.decode("string", Map("sem" -> "{b2}"))` There is an infinite number of sentences that could refer to button b2 and the decoding will never stop.